In [16]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

2024-05-20 23:14:57.971752: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 23:14:57.974181: I external/local_tsl/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2024-05-20 23:14:58.006027: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-05-20 23:14:58.807539: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [17]:
ham10000_metadata = pd.read_csv("dataset/HAM10000_metadata.csv")

ham10000_metadata.head(10)

,lesion_id,image_id,dx,dx_type,age,sex,localization
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back


In [18]:
IMAGE_NAME = 1
ILLNESS = 2
DIAGNOSTIC = 3
AGE = 4
SEX = 5
LOCALIZATION = 6

In [23]:
ham10000_metadata["age"].dropna()

0        80.0
1        80.0
2        80.0
3        80.0
4        75.0
         ... 
10010    40.0
10011    40.0
10012    40.0
10013    80.0
10014    70.0
Name: age, Length: 9958, dtype: float64

In [25]:
import os

from glob import glob

base_skin_dir = "dataset/HAM10000_images"

imageid_path_dict = {
    os.path.splitext(os.path.basename(x))[0]: x for x in glob(os.path.join(base_skin_dir, "*.jpg"))
}

ham10000_metadata["path"] = ham10000_metadata["image_id"].map(imageid_path_dict.get)

In [28]:
lesion_type_dict = {
    "nv": "Melanocytic nevi",
    "mel": "Melanoma",
    "bkl": "Benign keratosis-like lesions ",
    "bcc": "Basal cell carcinoma",
    "akiec": "Actinic keratoses",
    "vasc": "Vascular lesions",
    "df": "Dermatofibroma"
}

ham10000_metadata["cell_type"] = ham10000_metadata["dx"].map(lesion_type_dict.get)
ham10000_metadata["cell_type_idx"] = pd.Categorical(ham10000_metadata["cell_type"]).codes

In [29]:
ham10000_metadata.head(10)

,lesion_id,image_id,dx,dx_type,age,sex,localization,path,cell_type,cell_type_idx
0,HAM_0000118,ISIC_0027419,bkl,histo,80.0,male,scalp,dataset/HAM10000_images/ISIC_0027419.jpg,Benign keratosis-like lesions,2
1,HAM_0000118,ISIC_0025030,bkl,histo,80.0,male,scalp,dataset/HAM10000_images/ISIC_0025030.jpg,Benign keratosis-like lesions,2
2,HAM_0002730,ISIC_0026769,bkl,histo,80.0,male,scalp,dataset/HAM10000_images/ISIC_0026769.jpg,Benign keratosis-like lesions,2
3,HAM_0002730,ISIC_0025661,bkl,histo,80.0,male,scalp,dataset/HAM10000_images/ISIC_0025661.jpg,Benign keratosis-like lesions,2
4,HAM_0001466,ISIC_0031633,bkl,histo,75.0,male,ear,dataset/HAM10000_images/ISIC_0031633.jpg,Benign keratosis-like lesions,2
5,HAM_0001466,ISIC_0027850,bkl,histo,75.0,male,ear,dataset/HAM10000_images/ISIC_0027850.jpg,Benign keratosis-like lesions,2
6,HAM_0002761,ISIC_0029176,bkl,histo,60.0,male,face,dataset/HAM10000_images/ISIC_0029176.jpg,Benign keratosis-like lesions,2
7,HAM_0002761,ISIC_0029068,bkl,histo,60.0,male,face,dataset/HAM10000_images/ISIC_0029068.jpg,Benign keratosis-like lesions,2
8,HAM_0005132,ISIC_0025837,bkl,histo,70.0,female,back,dataset/HAM10000_images/ISIC_0025837.jpg,Benign keratosis-like lesions,2
9,HAM_0005132,ISIC_0025209,bkl,histo,70.0,female,back,dataset/HAM10000_images/ISIC_0025209.jpg,Benign keratosis-like lesions,2


In [31]:
from PIL import Image


ham10000_metadata["image"] = ham10000_metadata["path"].map(lambda x: np.asarray(Image.open(x).resize((200,150))))

In [40]:
from sklearn.model_selection import train_test_split

x = ham10000_metadata.drop(columns=["cell_type_idx"],axis=1)
y = ham10000_metadata["cell_type_idx"]

train_ratio = 0.70
validation_ratio = 0.20
test_ratio = 0.10

X_train, X_temp, Y_train, Y_temp = train_test_split(x, y, test_size=1 - train_ratio, random_state=1, stratify=y)

X_val, X_test, Y_val, Y_test = train_test_split(X_temp, Y_temp, test_size=test_ratio / (test_ratio + validation_ratio), random_state=1, stratify=Y_temp)

In [47]:
print("X_train shape:", X_train.shape)
print("X_val shape:", X_val.shape)
print("X_test shape:", X_test.shape)

# Verificando a distribuição das classes nos conjuntos de dados
print("Distribuição de classes em Y_train:", Y_train.nunique())
print("Distribuição de classes em Y_val:", Y_val.nunique())
print("Distribuição de classes em Y_test:", Y_test.nunique())

X_train shape: (7010, 10)
X_val shape: (2003, 10)
X_test shape: (1002, 10)
Distribuição de classes em Y_train: 7
Distribuição de classes em Y_val: 7
Distribuição de classes em Y_test: 7


In [48]:
Y_train_c = to_categorical(Y_train, num_classes = Y_train.nunique())
Y_val_c = to_categorical(Y_train, num_classes = Y_val.nunique())
Y_test_c = to_categorical(Y_train, num_classes = Y_test.nunique())
